In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./static/data/dist_info.csv')
df.head()

,구별,인구수,한국인,외국인,고령자,공원수,면적,1인당공원면적,공원면적합,공원면적비
0,종로구,159842,149952,9890,28396,12,23.91,24.447145,3.907681,16.343290
1,중구,135321,125800,9521,24265,5,9.96,21.824324,2.953289,29.651500
2,용산구,244953,229786,15167,39995,2,21.87,1.009149,0.247194,1.130289
3,성동구,302695,295591,7104,45372,5,16.86,2.070859,0.626839,3.717905
4,광진구,361923,348064,13859,50047,2,17.06,1.613608,0.584002,3.423224


In [3]:
import requests
from urllib.parse import urlparse, quote

In [4]:
key_fd = open('roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [5]:
road_addrs = []
dists = list(df['구별'])
for i in range(len(dists)):
    dists[i] += '청'
for dist in dists:
    road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
    url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&coutPerPage=10&keyword={quote(dist)}&resultType=json'
    result = requests.get(url).json()
    road_addr = result['results']['juso'][0]['roadAddr']
    road_addrs.append(road_addr)
road_addrs

['서울특별시 종로구 삼봉로 43 (수송동)',
 '부산광역시 중구 중구로 120 (대청동1가)',
 '서울특별시 용산구 백범로79길 64 (청파동3가)',
 '서울특별시 성동구 고산자로 270 (행당동)',
 '서울특별시 광진구 자양로 117 (자양동)',
 '서울특별시 동대문구 천호대로 145 (용두동)',
 '서울특별시 중랑구 봉화산로 179 (신내동)',
 '서울특별시 성북구 보문로 168 (삼선동5가)',
 '서울특별시 강북구 도봉로89길 13 (수유동)',
 '서울특별시 도봉구 마들로 656 (방학동)',
 '서울특별시 노원구 노해로 437 (상계동)',
 '서울특별시 은평구 은평로 195 (녹번동)',
 '서울특별시 서대문구 연희로 247 (연희동)',
 '서울특별시 마포구 월드컵로 212 (성산동)',
 '서울특별시 양천구 목동동로 105 (신정동)',
 '서울특별시 강서구 남부순환로 208 (외발산동)',
 '서울특별시 구로구 가마산로 245 (구로동)',
 '서울특별시 금천구 시흥대로73길 70 (시흥동)',
 '서울특별시 영등포구 당산로 123 (당산동3가)',
 '서울특별시 동작구 장승배기로 161 (노량진동)',
 '서울특별시 관악구 관악로 145 (봉천동)',
 '서울특별시 서초구 남부순환로 2584 (서초동)',
 '서울특별시 강남구 도곡로18길 7 (도곡동)',
 '서울특별시 송파구 올림픽로 326 (신천동)',
 '서울특별시 강동구 동남로 942 (고덕동)']

In [6]:
key_fd = open('kakaokey.txt', mode='r')
kakao_key = key_fd.read(100)
key_fd.close()

In [7]:
lat_list, lng_list = [], []
for addr in road_addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(addr)
    result = requests.get(url, headers={"Authorization": 'KakaoAK '+kakao_key}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat); lng_list.append(lng)

In [8]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head()

,구별,인구수,한국인,외국인,고령자,공원수,면적,1인당공원면적,공원면적합,공원면적비,위도,경도
0,종로구,159842,149952,9890,28396,12,23.91,24.447145,3.907681,16.343290,37.573199,126.978813
1,중구,135321,125800,9521,24265,5,9.96,21.824324,2.953289,29.651500,35.106197,129.032351
2,용산구,244953,229786,15167,39995,2,21.87,1.009149,0.247194,1.130289,37.541602,126.964977
3,성동구,302695,295591,7104,45372,5,16.86,2.070859,0.626839,3.717905,37.563423,127.036965
4,광진구,361923,348064,13859,50047,2,17.06,1.613608,0.584002,3.423224,37.538540,127.081912


In [9]:
df.to_csv('./static/data/dist_info.csv')

In [17]:
park_new = pd.read_csv('./static/data/park_info.csv')
df2 = park_new[park_new['구별'] == '양천구'].reset_index()
df2

,index,Unnamed: 0,공원번호,공원명,면적,구별,주소,경도,위도,공원개요,공원크기
0,30,30,47,계남근린공원,603715.2,양천구,서울특별시 양천구 신정동 산44-2일대,126.847964,37.513836,휴식공간 양천구 신정동과 구로구 고척동 일대의 옛지명인 부평군 계남면의 지명을 따서...,소
1,52,52,127,서서울호수공원,217946.0,양천구,서울특별시 양천구 남부순환로64길 20 (신월동),126.830479,37.527839,2009년 10월 개장한 서서울호수공원은 양천구 신월동에 위치한 옛 신월정수장을 공...,소
2,53,53,104,용왕산근린공원,213552.0,양천구,서울특별시 양천구 목동 199-51,126.878313,37.542959,옛 지도상에는 엄지산(嚴知山)으로 기록되어 있으며 엄지의 뜻은 순수한 우리말인 첫머...,소
3,67,67,61,갈산근린공원,138596.0,양천구,서울특별시 양천구 신정동 162-56,126.868915,37.508212,용왕산의 동쪽 능선 끝이 안양천에 닿아 있는데 이 안양천을 따라 남쪽으로 내려가서 ...,소
4,101,101,51,파리근린공원,29619.3,양천구,서울특별시 양천구 목동동로 363,126.876393,37.534974,프랑스와 수교 100주년이 되는 1986년에 기념 사업의 하나로 파리에 서울광장을 ...,소


In [18]:
df2['공원명'][4]

'파리근린공원'

In [19]:
for i in df2.index:
    if df2['공원크기'][i] == '대':
        print('대')
    elif df2['공원크기'][i] == '중':
        print('중')
    else:
        print('소')

소
소
소
소
소
